# Number of *de novo* consensi that have been masked or not

In [65]:
RM_out = "/beegfs/data/tbarberis/TE_alb/library_construction/RM_raw/GCA_006496715.1_Aalbo_primary.1_genomic.fna.out"
TE_lib_raw = "/beegfs/data/tbarberis/TE_alb/library_construction/TE_Aealb.raw.fa"
repbase_lib = "/beegfs/data/tbarberis/TE_alb/library_construction/raw_consensi/RepBase.fasta"
RM2_lib = "/beegfs/data/tbarberis/TE_alb/library_construction/raw_consensi/RM2_consensi.fa"
EDTA_lib = "/beegfs/data/tbarberis/TE_alb/annotation/de_novo/edta/GCA_006496715.1_Aalbo_primary.1_genomic.short_header.fna.mod.EDTA.TElib.fa"
MITE_lib = "/beegfs/data/tbarberis/TE_alb/library_construction/raw_consensi/MITE_consensi.fa"

## Parse `RepeatMasker` output

In [6]:
with open(RM_out, "r") as rmout:
    for i in range(3):
        rmout.readline()
    data = rmout.readlines()

In [8]:
matching_repeats = []
for line in data:
    match = line.split()[9]
    if match not in matching_repeats:
        matching_repeats.append(match)

In [10]:
len(matching_repeats)

38165

## Number of consensi in the raw library

In [12]:
import subprocess 

In [39]:
proc = subprocess.Popen(['grep', '-c', '>', TE_lib_raw], stdout=subprocess.PIPE)
nb_consensi = int(str(proc.stdout.read()).split("'")[1].split("\\")[0])
nb_consensi

59848

`RepeatMasker` has filtered 59848 - 38165 = 21683 putative sequences. Now we can se from which tool (`RepeatModeler2`, `EDTA` or `MITE-Tracker`) they come.

### RepBase

In [40]:
from Bio import SeqIO

In [54]:
repbase_cons = []
for seq_record in SeqIO.parse(repbase_lib, "fasta"):
    name = seq_record.description.split()[0]
    if name not in repbase_cons:
        repbase_cons.append(name)
len(repbase_cons)

17410

In [55]:
not_match_repbase = 0
for cons in repbase_cons:
    if cons not in matching_repeats:
        not_match_repbase += 1
not_match_repbase

12205

In [73]:
repbase_false_positive = (not_match_repbase * 100) / len(repbase_cons)
repbase_false_positive

70.10338885697875

### RepeatModeler2

In [58]:
RM2_cons = []
for seq_record in SeqIO.parse(RM2_lib, "fasta"):
    name = seq_record.description.split("#")[0]
    if name not in RM2_cons:
        RM2_cons.append(name)
len(RM2_cons)

15384

In [59]:
not_match_RM2 = 0
for cons in RM2_cons:
    if cons not in matching_repeats:
        not_match_RM2 += 1
not_match_RM2

6577

In [74]:
RM2_false_positive = (not_match_RM2 * 100) / len(RM2_cons)
RM2_false_positive

42.75221008840354

### EDTA

In [63]:
EDTA_cons = []
for seq_record in SeqIO.parse(EDTA_lib, "fasta"):
    name = seq_record.description.split("#")[0]
    if name not in EDTA_cons:
        EDTA_cons.append(name)
len(EDTA_cons)

16191

In [64]:
not_match_EDTA = 0
for cons in EDTA_cons:
    if cons not in matching_repeats:
        not_match_EDTA += 1
not_match_EDTA

2695

In [75]:
EDTA_false_positive = (not_match_EDTA * 100) / len(EDTA_cons)
EDTA_false_positive

16.64504971897968

### MITE-Tracker

In [68]:
MITE_cons = []
for seq_record in SeqIO.parse(MITE_lib, "fasta"):
    name = seq_record.description.split()[0]
    if name not in MITE_cons:
        MITE_cons.append(name)
len(MITE_cons)

10863

In [69]:
not_match_MITE = 0
for cons in MITE_cons:
    if cons not in matching_repeats:
        not_match_MITE += 1
not_match_MITE

206

In [76]:
MITE_false_positive = (not_match_MITE * 100) / len(MITE_cons)
MITE_false_positive

1.8963453926171407

### Plots

In [80]:
import plotly.express as px
import pandas as pd

In [109]:
data = {'Tool': ["RepBase", "RepeatModeler2", "EDTA", "MITE-Tracker"], '% of consensi not matching on the genome': \
        [repbase_false_positive, RM2_false_positive, EDTA_false_positive, MITE_false_positive]}
df = pd.DataFrame.from_dict(data)

fig = px.bar(df, x='Tool', y='% of consensi not matching on the genome', text_auto='.3s', width = 700)
fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
)
fig.update_traces(
    textposition="outside"
)
fig.show()

In [110]:
fig.write_image("nb_cons_by_tool.pdf")